## **Importing the Libraries**

In [1]:
# Import necessary libraries
from time import sleep  # Allows the program to pause for a specified amount of time
import pandas as pd  # Provides data manipulation and analysis tools
from selenium import webdriver  # Allows automated web browsing
from bs4 import BeautifulSoup  # Parses HTML and XML documents
from selenium.webdriver.common.by import By  # Provides a way to locate elements on a webpage
from selenium.webdriver.support.ui import WebDriverWait  # Allows the program to wait for an element to load before continuing
from selenium.webdriver.support import expected_conditions as EC  # Allows the program to specify expected conditions for an element to load
from datetime import date  # Provides tools for working with dates
from unidecode import unidecode  # Allows the program to remove accents and other diacritical marks from characters
import random  # Provides tools for generating random numbers and sequences
import datetime  # Provides tools for working with dates and times

## **Scrapping the Data**

### *Decolar Function*

The function **scrape_decolar_data()** uses web scraping techniques to collect data about flights from the Decolar website.

The function takes the following inputs:

- ``from_location``: the departure airport or city (string)
- ``to_location``: the arrival airport or city (string)
- ``departure_date``: the date of departure (string in the format "YYYY-MM-DD")
- ``arrival_date``: the date of arrival (string in the format "YYYY-MM-DD")
- ``adult_qty``: the number of adults (integer)

And return as an output an Pandas DataFrame with all the flights information scrapped from the website

In [51]:
def scrape_decolar_data(from_location, to_location, departure_date, arrival_date, adult_qty):
    # Initialize search day, search ID and company name
    search_day = date.today()
    search_id = str(search_day) + from_location + to_location + departure_date + arrival_date
    company = 'Decolar'

    # Set up web driver options and open the search URL
    options = webdriver.ChromeOptions()
    options.add_argument('--headless') # Allows to run automated scripts in headless mode, meaning that the browser window wouldn't be visible.
    with webdriver.Chrome(options=options) as driver: # Initializing webdriver
        driver.maximize_window()
        search_Url = 'https://www.decolar.com/shop/flights/results/roundtrip/{fromLocation}/{toLocation}/{departureDate}/{arrivalDate}/1/0/0/NA/NA/NA/NA/NA?from=SB&di={adultQty}-0&reSearch=true'\
        .format(fromLocation = from_location, 
                toLocation = to_location, 
                departureDate = departure_date, 
                arrivalDate = arrival_date, 
                adultQty = adult_qty
                )
        driver.get(search_Url)
        
        # Set up WebDriverWait
        wait = WebDriverWait(driver, 10)

        # Closing popup
        popup_window = '//*[@id="dreck-wrongcountry-modal"]/div[1]/i'
        try:
            wait.until(EC.element_to_be_clickable((By.XPATH, popup_window))).click() 
        except:
            print('No popup window')
        
        # Closing discount popup
        popup_discount = '//*[@id="header"]/nav/div[6]/div[1]/i'
        try:
            wait.until(EC.element_to_be_clickable((By.XPATH, popup_discount))).click() 
        except:
            print("No discount popup")

        # Sorting By Cheapest
        relXpathSortByCheapestButton = '//*[@id="flights-container"]/div/div[3]/div/div[2]/div/div[4]/app-root/app-common/new-sorting-tabs/div/tab-component[2]'
        try: 
            wait.until(EC.element_to_be_clickable((By.XPATH, relXpathSortByCheapestButton))).click()
        except:
            print("No sort by Cheapest Button")

        # Wating until flight containers are present on the page before proceeding
        wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="cluster-container COMMON"]'))) 
        
        # Scrooling down the page to load more results, and see if the button "load more flights has appeared"
        for i in range(0,5):
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            sleep(1)

        # Seeing if the "load more button has appeared" and assigning a True or False value to the variable
        more_results_button = '//body/div/div/div/div/div/div/div/div/div/app-root/app-common/items/div/div/a[1]'
        try:
            status_search_see_more_flights_button = wait.until(EC.element_to_be_clickable((By.XPATH, more_results_button))).is_displayed()
        except:
            status_search_see_more_flights_button = False

        # While loop to load more fights until the button "load more flights" disappears
        while status_search_see_more_flights_button == True:
            try:
                print("Loading more flights")
                driver.execute_script("window.scrollTo(0,document.body.scrollHeight)") # Scrool down te page
                wait.until(EC.element_to_be_clickable((By.XPATH, more_results_button))).click() # # Clicking the load more button when it loads
                status_search_see_more_flights_button = wait.until(EC.element_to_be_clickable((By.XPATH, more_results_button))).is_displayed() # Checking the status of the "load more flights"
            except:
                print('No more flights to load')
                break

        # Collecting Data
        try:
            flight_containers = driver.find_elements(By.XPATH, '//div[@class = "cluster-container COMMON"]')

            flight_list = []

            for WebElement in flight_containers:
                elementHTML = WebElement.get_attribute('outerHTML')
                elementSoup = BeautifulSoup(elementHTML, 'html.parser')

                # Dictionary to store the flight card data
                flight_data = {}

                # SearchID, SearchDay, and serachUrl
                flight_data['searchID'] = search_id
                flight_data['searchDay'] = search_day
                flight_data['searchUrl'] = search_Url
                flight_data['departureDate'] = departure_date
                flight_data['arrivalDate'] = arrival_date
                flight_data['adultQty'] = adult_qty
                flight_data['company'] = company
                
                # Origin Airport and Destiny Airport
                flight_data['originAirport'] = elementSoup.find('span', class_ = 'route-location route-departure-location').text.strip().split(' ', 1)[0]
                flight_data['destinyAirport'] = elementSoup.find('span', class_ = 'route-location route-arrival-location').text.strip().split(' ', 1)[0]
                
                # # Value Tarif and Taxes
                flight_data['tarif'] = 0
                flight_data['taxes'] = 0

                # Currency and Price
                flight_data['currency'] = elementSoup.find('span', class_ = 'currency price-mask -eva-3-mr-xsm').text
                flight_data['value'] = elementSoup.find('span', class_ = 'amount price-amount').text.replace('.','').strip()

                # CIA and CIA_abv
                airline_img_container = elementSoup.find('span', class_='container-img-airlines')
                list_cia_flight_container = [] # List to store the companies name, because some of them have more than 1
                list_cia_abv_flight_container = [] # List to store the abreviated companies name, because some of them have more than 1
                for img in airline_img_container.find_all('img'):
                    list_cia_flight_container.append(unidecode(img['alt'].strip()))
                    list_cia_abv_flight_container.append(unidecode(img['alt'].strip()[:4]))
                flight_data['cia'] = list_cia_flight_container
                flight_data['cia_abv'] = list_cia_abv_flight_container

                flight_list.append(flight_data)

            dataTypeDict = {"searchID" : 'object', 'searchDay' : 'datetime64[ns]', "originAirport" : 'object', "destinyAirport" : 'object', 
                        "searchUrl": 'object', "departureDate" : 'datetime64[ns]', "arrivalDate" : 'datetime64[ns]', "adultQty" : 'int64', 
                        "company" : 'object', "cia" : 'object', 'cia_abv' : 'object', "currency" : 'object', "tarif" : 'float32', "taxes" : 'float32', 
                        "value" : 'float32'}
            # Creating the Data Frame
            df = pd.DataFrame(flight_list)
            # # Reordering the columns
            cols = list(dataTypeDict.keys())
            df = df[cols]
            # # Changing the columns type
            df = df.astype(dtype=dataTypeDict).sort_values('value', ascending=True)
            ## Exploding the columns cia and cia_abv, because some of them have more than 1 cia separated by ","
            df = df.explode(['cia', 'cia_abv'], ignore_index=True).sort_values('value', ascending=True)
            print(f'{df.shape[0]} flights were scrapped from decolar')

        except:
            print('No flight containers found')
 
    return df

In [3]:
# Example
from_location = 'GRU'
to_location = 'JFK'
departure_date = '2023-08-01'
arrival_date = '2023-08-10'
adult_qty = '1'

decolar_df = scrape_decolar_data(from_location, to_location, departure_date, arrival_date, adult_qty)
decolar_df.head()

No discount popup
Loading more flights
Loading more flights
Loading more flights
Loading more flights
Loading more flights
Loading more flights
Loading more flights
No more flights to load
102 flights were scrapped from decolar


,searchID,searchDay,originAirport,destinyAirport,searchUrl,departureDate,arrivalDate,adultQty,company,cia,cia_abv,currency,tarif,taxes,value
0,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.decolar.com/shop/flights/results/r...,2023-08-01,2023-08-10,1,Decolar,LATAM,LATA,R$,0.0,0.0,3030.0
1,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.decolar.com/shop/flights/results/r...,2023-08-01,2023-08-10,1,Decolar,LATAM,LATA,R$,0.0,0.0,3053.0
2,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.decolar.com/shop/flights/results/r...,2023-08-01,2023-08-10,1,Decolar,LATAM,LATA,R$,0.0,0.0,3081.0
3,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.decolar.com/shop/flights/results/r...,2023-08-01,2023-08-10,1,Decolar,Delta Air Lines,Delt,R$,0.0,0.0,3081.0
4,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.decolar.com/shop/flights/results/r...,2023-08-01,2023-08-10,1,Decolar,LATAM,LATA,R$,0.0,0.0,3081.0


### *Passagens Promo Function*

The function **scrape_decolar_data()** uses web scraping techniques to collect data about flights from the Decolar website. The logic is very similar to the first function.

The function takes the following inputs:

- ``from_location``: the departure airport or city (string)
- ``to_location``: the arrival airport or city (string)
- ``departure_date``: the date of departure (string in the format "YYYY-MM-DD")
- ``arrival_date``: the date of arrival (string in the format "YYYY-MM-DD")
- ``adult_qty``: the number of adults (integer)

And return as an output an Pandas DataFrame with all the flights information scrapped from the website

In [49]:
def scrape_passagens_promo_data(from_location, to_location, departure_date, arrival_date, adult_qty):
    # Initialize search day, search ID and company name
    search_day = date.today()
    search_id = str(search_day) + from_location + to_location + departure_date + arrival_date
    company = 'PP'

    # Set up web driver options and open the search URL
    options = webdriver.ChromeOptions()
    options.add_argument('--headless') # Allows to run automated scripts in headless mode, meaning that the browser window wouldn't be visible.
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-gpu")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36")
    with webdriver.Chrome(options=options) as driver:
        search_Url = 'https://www.passagenspromo.com.br/air/search/{fromLocation}{toLocation}{departureDate}-{toLocation}{fromLocation}{arrivalDate}/{adultQty}/0/0/Y/?'\
        .format(
            fromLocation = from_location, 
            toLocation = to_location, 
            departureDate = departure_date.replace('-', '')[2:], 
            arrivalDate = arrival_date.replace('-', '')[2:], 
            adultQty = adult_qty
        )
        driver.get(search_Url)

        # Set up WebDriverWait
        wait = WebDriverWait(driver, 15)

        # Wait for flight cards to load
        flight_card = '//div[@class="flightgroupcard"]'
        try:
            wait.until(EC.visibility_of_all_elements_located((By.XPATH, flight_card)))
        except:
            print('Not all flight cards were visible')
        
        # Check if "load more" button is visible
        more_results_button = '//button[normalize-space()="Mais resultados"]'
        try:
            status_search_see_more_flights_button = wait.until(EC.element_to_be_clickable((By.XPATH, more_results_button))).is_displayed()
        except:
            print('No load more flights button was found')
            status_search_see_more_flights_button = False
        
        # Load more flights while load more flights button is displayed
        while status_search_see_more_flights_button == True:
            try:
                print("Loading more flights")
                driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
                wait.until(EC.element_to_be_clickable((By.XPATH, more_results_button))).click()
                status_search_see_more_flights_button = wait.until(EC.element_to_be_clickable((By.XPATH, more_results_button))).is_displayed()
            except:
                print('The load more flights button has dissapeared. No more flights to load')
                break

        # Collecting the Data
        try:
            fligths_rows = driver.find_elements(By.CLASS_NAME, "flightcard")
            flight_list = []

            for WebElement in fligths_rows:
                elementHTML = WebElement.get_attribute('outerHTML')
                elementSoup = BeautifulSoup(elementHTML, 'html.parser')

                # Dictionary to store the flight card data
                flight_data = {}

                # SearchID, SearchDay, and serachUrl
                flight_data['searchID'] = search_id
                flight_data['searchDay'] = search_day
                flight_data['searchUrl'] = search_Url
                flight_data['departureDate'] = departure_date
                flight_data['arrivalDate'] = arrival_date
                flight_data['adultQty'] = adult_qty
                flight_data['company'] = company
                
                # Origin Airport and Destiny Airport
                flight_data['originAirport'] = elementSoup.find('label', class_ = 'origin_iata').text
                flight_data['destinyAirport'] = elementSoup.find('label', class_ = 'destiny_iata').text
                
                # Value Tarif and Taxes
                flight_data['tarif'] = elementSoup.find('span', 'value_passenger').text.split(' ', 1)[1].replace('.','')
                flight_data['taxes'] = elementSoup.find('span', 'value_tax').text.split(' ', 1)[1].replace('.','')

                # Currency and Price
                flight_data['currency'] = elementSoup.find('div', class_ = 'total_price').text.split(' ', 1)[0]
                flight_data['value'] = elementSoup.find('div', class_ = 'total_price').text.split(' ', 1)[1].replace('.','')

                # CIA and CIA_abv
                flight_data['cia'] = elementSoup.find('div', class_='logo_cia').text.strip()
                flight_data['cia_abv'] = elementSoup.find('div', class_='logo_cia').text.strip()[:4]

                flight_list.append(flight_data)

            dataTypeDict = {"searchID" : 'object', 'searchDay' : 'datetime64[ns]', "originAirport" : 'object', "destinyAirport" : 'object', 
                        "searchUrl": 'object', "departureDate" : 'datetime64[ns]', "arrivalDate" : 'datetime64[ns]', "adultQty" : 'int64', 
                        "company" : 'object', "cia" : 'object', 'cia_abv' : 'object', "currency" : 'object', "tarif" : 'float32', "taxes" : 'float32', 
                        "value" : 'float32'}
            # Creating the Data Frame
            df = pd.DataFrame(flight_list)
            # # Reordering the columns
            cols = list(dataTypeDict.keys())
            df = df[cols]
            # # Changing the columns type
            df = df.astype(dtype=dataTypeDict).sort_values('value', ascending=True)
            print(f'{df.shape[0]} flights were scrapped from Passagens Promo')

        except:
            print('No flight cards found')
        
    return df

In [ ]:
# Example
from_location = 'GRU'
to_location = 'JFK'
departure_date = '2023-08-01'
arrival_date = '2023-08-10'
adult_qty = '1'

passagens_promo_df = scrape_passagens_promo_data(from_location, to_location, departure_date, arrival_date, adult_qty)
passagens_promo_df.head()

Loading more flights
Loading more flights
Loading more flights
Loading more flights
Loading more flights
Loading more flights
Loading more flights
Loading more flights
Loading more flights
Loading more flights
The load more flights button has dissapeared. No more flights to load
82 flights were scrapped from Passagens Promo


,searchID,searchDay,originAirport,destinyAirport,searchUrl,departureDate,arrivalDate,adultQty,company,cia,cia_abv,currency,tarif,taxes,value
0,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.passagenspromo.com.br/air/search/G...,2023-08-01,2023-08-10,1,PP,LATAM,LATA,R$,2456.0,573.0,3030.0
1,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.passagenspromo.com.br/air/search/G...,2023-08-01,2023-08-10,1,PP,LATAM,LATA,R$,2456.0,573.0,3030.0
2,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.passagenspromo.com.br/air/search/G...,2023-08-01,2023-08-10,1,PP,LATAM,LATA,R$,2456.0,596.0,3052.0
3,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.passagenspromo.com.br/air/search/G...,2023-08-01,2023-08-10,1,PP,LATAM,LATA,R$,2456.0,596.0,3052.0
4,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.passagenspromo.com.br/air/search/G...,2023-08-01,2023-08-10,1,PP,LATAM,LATA,R$,2456.0,596.0,3052.0


### *Merging both Functions*

In [4]:
def merged_scraper(from_location, to_location, departure_date, arrival_date, adult_qty):
    decolar_df = scrape_decolar_data(from_location, to_location, departure_date, arrival_date, adult_qty)
    passagens_promo_df = scrape_passagens_promo_data(from_location, to_location, departure_date, arrival_date, adult_qty)
    return decolar_df, passagens_promo_df

## **Formating the Scrapped Data**

Our objective with this project is to be able to compare Decolar to Passagens Promo. So we must format and clean our DataFrame to use in PowerBI to further analyze how each website compare to each other.

So first we need to transform, clean and organize the data from the **decolar_df** and **passagens_promo_df**. We are going to:
- Filter the dataframe to only display the cheapest price per `cia` among the website;
- Create a column called ``bestCia`` to identify the best `cia` among the website;
- Create a columns caled `comparisonWithPP` to store the comparison between Decolar and Passagens Promo;
- Merge both dataframes in our **final_df**

### *passagens_promo_df*

In [11]:
# Transforming the passagens_promo_df
## Filtering the DataFrame to only display the cheapest price per cia
bestPricesPerCiaPP = passagens_promo_df.groupby('cia').min().sort_values('value', ascending=True).reset_index()
## Geting the index of the flight with the lowest value
best_cia = bestPricesPerCiaPP.loc[bestPricesPerCiaPP['value'].idxmin(), 'value'] 
## Creating 'bestCia' column with 1 for the 'cia' with the lowest value of all, and 0 otherwise
bestPricesPerCiaPP['bestCia'] = bestPricesPerCiaPP['value'].apply(lambda x: 1 if x == best_cia else 0)
## Creating 'comparisonWithPP'. We put 'PP X PP', because we would be comparing Passagens Promo with itself
bestPricesPerCiaPP['comparisonWithPP'] = 'PP X PP'
bestPricesPerCiaPP.head()

,cia,searchID,searchDay,originAirport,destinyAirport,searchUrl,departureDate,arrivalDate,adultQty,company,cia_abv,currency,tarif,taxes,value,comparisonWithPP,bestCia
0,LATAM,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.passagenspromo.com.br/air/search/G...,2023-08-01,2023-08-10,1,PP,LATA,R$,2456.0,573.0,3030.0,PP X PP,1
1,American Airlines,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.passagenspromo.com.br/air/search/G...,2023-08-01,2023-08-10,1,PP,Amer,R$,3268.0,652.0,3920.0,PP X PP,0
2,Delta,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.passagenspromo.com.br/air/search/G...,2023-08-01,2023-08-10,1,PP,Delt,R$,3464.0,505.0,3969.0,PP X PP,0
3,Avianca,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.passagenspromo.com.br/air/search/G...,2023-08-01,2023-08-10,1,PP,Avia,R$,4116.0,690.0,4805.0,PP X PP,0
4,Copa,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.passagenspromo.com.br/air/search/G...,2023-08-01,2023-08-10,1,PP,Copa,R$,4060.0,757.0,4817.0,PP X PP,0


### *decolar_df*

In [12]:
# Transforming the decolar_df
## Filtering the DataFrame to only display the cheapest price per cia
bestPricesPerCiaDecolar = decolar_df.groupby('cia').min().sort_values('value', ascending=True).reset_index()
## Geting the index of the flight with the lowest value
best_cia = bestPricesPerCiaDecolar.loc[bestPricesPerCiaDecolar['value'].idxmin(), 'value'] 
## Creating 'bestCia' column with 1 for the 'cia' with the lowest value of all, and 0 otherwise
bestPricesPerCiaDecolar['bestCia'] = bestPricesPerCiaDecolar['value'].apply(lambda x: 1 if x == best_cia else 0)

# Create the column 'comparisonWithPP', which compare the price from 'decolar' and 'passagens promo' for the same cia 
for indexD, rowD in bestPricesPerCiaDecolar.iterrows():
    for indexP, rowP in bestPricesPerCiaPP.iterrows():
        if rowD.cia_abv == rowP.cia_abv:
            if rowD.value > rowP.value:
                bestPricesPerCiaDecolar.loc[indexD,'comparisonWithPP'] = 'Win / Cheaper'
            else:
                bestPricesPerCiaDecolar.loc[indexD,'comparisonWithPP'] = 'Loss / Expensiver'
                    
# Filling na values in case 'passagens_promo_df' did not have a CIA that decolar_df had
bestPricesPerCiaDecolar['comparisonWithPP'].fillna('Loss / CIA not returned', inplace = True) 
bestPricesPerCiaDecolar.head()

,cia,searchID,searchDay,originAirport,destinyAirport,searchUrl,departureDate,arrivalDate,adultQty,company,cia_abv,currency,tarif,taxes,value,bestCia,comparisonWithPP
0,LATAM,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.decolar.com/shop/flights/results/r...,2023-08-01,2023-08-10,1,Decolar,LATA,R$,0.0,0.0,3030.0,1,PERDA / Mais Barato
1,Delta Air Lines,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.decolar.com/shop/flights/results/r...,2023-08-01,2023-08-10,1,Decolar,Delt,R$,0.0,0.0,3596.0,0,PERDA / Mais Barato
2,American Airlines,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.decolar.com/shop/flights/results/r...,2023-08-01,2023-08-10,1,Decolar,Amer,R$,0.0,0.0,3999.0,0,GANHO / Mais Barato
3,Gol,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.decolar.com/shop/flights/results/r...,2023-08-01,2023-08-10,1,Decolar,Gol,R$,0.0,0.0,4673.0,0,PERDA / Mais Barato
4,Copa Airlines,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.decolar.com/shop/flights/results/r...,2023-08-01,2023-08-10,1,Decolar,Copa,R$,0.0,0.0,4699.0,0,PERDA / Mais Barato


### *Concatenating both Dataframes*

In [13]:
final_df = pd.concat([bestPricesPerCiaPP, bestPricesPerCiaDecolar], axis=0).reset_index().drop(columns='index')
final_df.head()

,cia,searchID,searchDay,originAirport,destinyAirport,searchUrl,departureDate,arrivalDate,adultQty,company,cia_abv,currency,tarif,taxes,value,comparisonWithPP,bestCia
0,LATAM,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.passagenspromo.com.br/air/search/G...,2023-08-01,2023-08-10,1,PP,LATA,R$,2456.0,573.0,3030.0,PP X PP,1
1,American Airlines,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.passagenspromo.com.br/air/search/G...,2023-08-01,2023-08-10,1,PP,Amer,R$,3268.0,652.0,3920.0,PP X PP,0
2,Delta,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.passagenspromo.com.br/air/search/G...,2023-08-01,2023-08-10,1,PP,Delt,R$,3464.0,505.0,3969.0,PP X PP,0
3,Avianca,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.passagenspromo.com.br/air/search/G...,2023-08-01,2023-08-10,1,PP,Avia,R$,4116.0,690.0,4805.0,PP X PP,0
4,Copa,2023-04-12GRUJFK2023-08-012023-08-10,2023-04-12,GRU,JFK,https://www.passagenspromo.com.br/air/search/G...,2023-08-01,2023-08-10,1,PP,Copa,R$,4060.0,757.0,4817.0,PP X PP,0


### *Creating a function to transform the data*

In [32]:
def transform_data(passagens_promo_df, decolar_df):

    # Transforming the passagens_promo_df
    bestPricesPerCiaPP = passagens_promo_df.groupby('cia').min().sort_values('value', ascending=True).reset_index()
    best_cia = bestPricesPerCiaPP.loc[bestPricesPerCiaPP['value'].idxmin(), 'value'] 
    bestPricesPerCiaPP['bestCia'] = bestPricesPerCiaPP['value'].apply(lambda x: 1 if x == best_cia else 0)
    bestPricesPerCiaPP['comparisonWithPP'] = 'PP X PP'

    # Transforming the decolar_df
    bestPricesPerCiaDecolar = decolar_df.groupby('cia').min().sort_values('value', ascending=True).reset_index()
    best_cia = bestPricesPerCiaDecolar.loc[bestPricesPerCiaDecolar['value'].idxmin(), 'value'] 
    bestPricesPerCiaDecolar['bestCia'] = bestPricesPerCiaDecolar['value'].apply(lambda x: 1 if x == best_cia else 0)

    for indexD, rowD in bestPricesPerCiaDecolar.iterrows():
        for indexP, rowP in bestPricesPerCiaPP.iterrows():
            if rowD.cia_abv == rowP.cia_abv:
                if rowD.value > rowP.value:
                    bestPricesPerCiaDecolar.loc[indexD,'comparisonWithPP'] = 'Win / Cheaper'
                else:
                    bestPricesPerCiaDecolar.loc[indexD,'comparisonWithPP'] = 'Loss / Expensiver'
                    
    bestPricesPerCiaDecolar['comparisonWithPP'].fillna('Loss / CIA not returned', inplace = True) 

    # Concatenating both Dataframes
    final_df = pd.concat([bestPricesPerCiaPP, bestPricesPerCiaDecolar], axis=0).reset_index().drop(columns='index')

    return final_df

### *Creating a function to Storage the data*

In [6]:
def load_data_to_spreadsheet(final_df):
    # Loading the Spreadsheet that Storage the Scrapped Flight Data
    fligh_data_worksheet = pd.read_excel('Flight Data.xlsx')

    # Appending the Data from the final_df into the fligh_data_worksheet if the searchID is not in the fligh_data_worksheet
    if final_df.searchID.unique() not in fligh_data_worksheet.searchID.unique():
        updated_fligh_data_worksheet = pd.concat([fligh_data_worksheet, final_df], axis=0).reset_index().drop(columns='index')
        updated_fligh_data_worksheet.to_excel('Flight Data.xlsx', index=False)
        print("Data appended into the spredsheet")
    else:
        print("Data not appended into the spredsheet")

## **Generating random url Parameters to use in the function**

Now that we have the web scrapping functions ready we need to create random values to use as parameters in the function.

In [ ]:
# Code to generate random values to use as parameters to form our webscrapping URL
START_DATE = datetime.date(2023, 5, 1)
END_DATE = datetime.date(2023, 10, 31)
NUM_SEARCHES = 300
def generate_random_search_params():
    # Dictionary to store the random search parameters data
    random_search_parameter_data = {}

    # Generating Random Departure location and Arrival Locations Airport
    iata_list = pd.read_excel('Dim_iata.xlsx').iata.unique()
    random_index0, random_index1 = random.randint(0, len(iata_list) - 1), random.randint(0, len(iata_list) - 1)
    random_search_parameter_data['Departure Iata'] = iata_list[random_index0]
    random_search_parameter_data['Arrival Iata'] = iata_list[random_index1]

    # Generating Random Departure Dates
    random_number_of_days = random.randint(0, (END_DATE - START_DATE).days)
    random_departure_date = START_DATE + datetime.timedelta(days=random_number_of_days)
    random_search_parameter_data['Random Departure Dates'] = random_departure_date

    # Generating Random Arrival Dates (Between 7 and 31 days from the departure)
    random_number_of_days = random.randint(7,31)
    random_arrival_date = random_departure_date + datetime.timedelta(days=random_number_of_days)
    random_search_parameter_data['Random Arrival Dates'] = random_arrival_date

    # Generating Random Adult Quantity from 1 to 4
    random_search_parameter_data['Random Adult Quantity'] = random.randint(1,4)

    # Returning the "random_search_parameter_data" dictionary
    return random_search_parameter_data

flight_search_parameters_list = [generate_random_search_params() for i in range(NUM_SEARCHES)]
flight_search_parameters_df = pd.DataFrame(flight_search_parameters_list)
flight_search_parameters_df.to_csv('random_generated_search_parameters.csv', index = False)

In [ ]:
flight_search_parameters_df.head()

,Departure Iata,Arrival Iata,Random Departure Dates,Random Arrival Dates,Random Adult Quantity
0,BZB,APK,2023-10-02,2023-10-25,1
1,WAE,PMD,2023-10-10,2023-11-07,1
2,YBB,ATV,2023-08-14,2023-08-25,3
3,LYR,AAE,2023-05-20,2023-05-27,1
4,ALG,LUR,2023-09-04,2023-09-18,4


## **Executing the functions and Loading results in an Excel File**

In [53]:
# Importing a spreadsheet with search parameters
search_parameters_df = pd.read_excel('search_parameters.xlsx')
search_parameters_df['Departure Dates'] = pd.to_datetime(search_parameters_df['Departure Dates']).dt.date
search_parameters_df['Arrival Dates'] = pd.to_datetime(search_parameters_df['Arrival Dates']).dt.date
print(f'There is {search_parameters_df.shape[0]} rows of search parameters')
search_parameters_df.head(1)

There is 42 rows of search parameters


,Departure Iata,Arrival Iata,Departure Dates,Arrival Dates,Adult Quantity
0,BSB,LIS,2023-10-02,2023-10-25,1


In [54]:
search_parameters_df = search_parameters_df.loc[37:].reset_index(drop=True)
search_parameters_df

,Departure Iata,Arrival Iata,Departure Dates,Arrival Dates,Adult Quantity
0,RIO,SAN,2023-10-05,2023-10-28,2
1,AJU,MCO,2023-07-13,2023-07-20,4
2,BSB,BOS,2023-07-19,2023-08-14,2
3,GRU,JFK,2023-05-16,2023-06-12,1
4,GIG,NRT,2023-06-11,2023-06-29,1


In [ ]:
for i in range(len(search_parameters_df)):
    
    # Getting the parameters for the 'search_parameters_df'
    from_location = search_parameters_df['Departure Iata'].loc[i]
    to_location = search_parameters_df['Arrival Iata'].loc[i]
    departure_date = str(search_parameters_df['Departure Dates'].loc[i])
    arrival_date = str(search_parameters_df['Arrival Dates'].loc[i])
    adult_qty = search_parameters_df['Adult Quantity'].loc[i]

    # Executing function to scrape the data
    decolar_df, passagens_promo_df = merged_scraper(from_location, to_location, departure_date, arrival_date, adult_qty)
    # Executing function to transform the data
    final_df = transform_data(passagens_promo_df, decolar_df) 
    # Loading the data into the spreadsheet
    load_data_to_spreadsheet(final_df)